MNIST 데이터셋을 활용하여 영화 추천 시스템 만들기
- 별점: 시청횟수
- 3점 미만의 데이터는 선호하지 않음으로 가정

### 1. 데이터 준비와 전처리

In [1]:
# 파일을 불러와서 앞의 5개의 데이터만 뽑아보기
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head() 

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남기고, 필터를 거친 데이터의 갯수와 비율 파악하기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꾸기
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 불러오기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# 데이터 프레임 만들기: user id, movie id, count, timestamp, title 순
data = ratings.join(movies.set_index('movie_id'), on='movie_id')
data.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [6]:
# user_id 10이 어떤 영화를 시청했는지 알아보기
movie_id_user10 = data[data['user_id']== 10]['title']
movie_id_user10

799                    Midsummer Night's Dream, A (1999)
800                           Mission: Impossible (1996)
801     Star Wars: Episode I - The Phantom Menace (1999)
802                           Defending Your Life (1991)
803                                 Breaking Away (1979)
                              ...                       
1195                             Far Off Place, A (1993)
1196                      Flight of the Navigator (1986)
1197                            Gnome-Mobile, The (1967)
1198                                Graduate, The (1967)
1199                    Happiest Millionaire, The (1967)
Name: title, Length: 389, dtype: object

### 2. 분석
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [7]:
print('ratings에 있는 유니크한 영화 개수: ', ratings['movie_id'].nunique())
print('ratings에 있는 유니크한 사용자 수 : ', ratings['user_id'].nunique())

ratings에 있는 유니크한 영화 개수:  3628
ratings에 있는 유니크한 사용자 수 :  6039


In [8]:
print('가장 인기 있는 영화 30개(인기순)')
ratings = ratings.merge(movies, how='left', on='movie_id')
ratings.groupby('title')['user_id'].count().sort_values(ascending=False)[:30]

가장 인기 있는 영화 30개(인기순)


title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 3. 내가 선호하는 영화를 5가지 골라서 ratings에 추가

In [9]:
# 제목으로 찾아보기
movies[movies['title'].str.lower().str.contains('batman', regex=False)]

,movie_id,title,genre
151,153,Batman Forever (1995),Action|Adventure|Comedy|Crime
588,592,Batman (1989),Action|Adventure|Crime|Drama
1356,1377,Batman Returns (1992),Action|Adventure|Comedy|Crime
1522,1562,Batman & Robin (1997),Action|Adventure|Crime
3144,3213,Batman: Mask of the Phantasm (1993),Animation|Children's


In [13]:
# 선호하는 5개의 영화 지정
my_favorite = ['Sixth Sense, The (1999)', 'batman (1989)', 'Matrix, The (1999)',
              'Men in Black (1997)', 'Toy Story (1995)']

In [14]:
# 나의 user_id를 만들기 위해 ratings의 마지막 5줄 확인
ratings.tail()

,user_id,movie_id,count,timestamp,title,genre
836473,6040,1090,3,956715518,Platoon (1986),Drama|War
836474,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096,4,956715648,Sophie's Choice (1982),Drama
836477,6040,1097,4,956715569,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi


In [15]:
# 나의 user_id를 6041로 만들어 5개의 영화 넣기
my_list = pd.DataFrame({'user_id': ['6041']*5, 'title': my_favorite, 'count': [5]*5})

if not ratings.isin({'user_id': ['6041']})['user_id'].any():
    ratings = ratings.append(my_list)

ratings.tail(10)

,user_id,movie_id,count,timestamp,title,genre
836473,6040,1090.0,3,956715518.0,Platoon (1986),Drama|War
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982),Drama
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,6041,NaN,5,NaN,"Sixth Sense, The (1999)",NaN
1,6041,NaN,5,NaN,batman (1989),NaN
2,6041,NaN,5,NaN,"Matrix, The (1999)",NaN
3,6041,NaN,5,NaN,Men in Black (1997),NaN
4,6041,NaN,5,NaN,Toy Story (1995),NaN


### 4. CSR matrix

In [16]:
# CSR Matrix 만들기
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,timestamp,title,genre
0,0,1193.0,5,978300760.0,0,Drama
1,0,661.0,3,978302109.0,1,Animation|Children's|Musical
2,0,914.0,3,978301968.0,2,Musical|Romance
3,0,3408.0,4,978300275.0,3,Drama
4,0,2355.0,5,978824291.0,4,Animation|Children's|Comedy
...,...,...,...,...,...,...
0,6039,NaN,5,NaN,38,NaN
1,6039,NaN,5,NaN,3628,NaN
2,6039,NaN,5,NaN,124,NaN
3,6039,NaN,5,NaN,175,NaN


In [17]:
from scipy.sparse import csr_matrix

#data column 값을 indexing 값으로 교체
#user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱
#movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [18]:
# Implicit Data는 유저가 간접적(Implicit)으로 선호, 취향을 나타내는 데이터를 의미한다
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 권장 세팅
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.01, 
                                    use_gpu=False, 
                                    iterations=15, 
                                    dtype=np.float32)

In [19]:
# 모델 Matrix transpose
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [26]:
preference, sixthsense = user_to_idx['6041'], movie_to_idx['Sixth Sense, The (1999)']
preference_vector, sixthsense_vector = als_model.user_factors[preference], als_model.item_factors[sixthsense]

In [27]:
preference_vector

array([ 0.7009415 , -0.7581013 ,  0.21781169,  0.24175513, -0.25984684,
        0.07331309, -0.5337341 ,  0.19631575,  0.02664329,  0.6300258 ,
       -1.3620044 , -0.6988337 , -0.5363199 , -0.23079415,  0.47954074,
        0.3201878 ,  0.19655864,  0.10880958,  0.4492817 , -0.15586466,
        0.5226364 , -0.8615651 , -0.32082286,  0.23716785, -0.30980042,
        0.23278026,  0.04232532,  0.5421749 ,  0.37239608, -0.00742769,
        0.21391828,  0.05516662, -0.10693467, -0.775651  ,  0.25155652,
       -0.2994681 ,  1.1272333 , -0.32485056,  0.0464655 , -0.70566744,
        1.0516952 , -0.89811665, -0.06442402, -0.45431423,  0.25374562,
       -0.3838175 ,  0.48926592,  0.05258114,  0.37516004, -0.7852077 ,
        0.6161453 ,  0.2651864 , -0.4773746 ,  0.3039889 , -0.31702864,
        0.80215484,  0.16299044,  0.06679183,  0.06692396, -0.2533681 ,
        0.35864758,  0.4134138 , -0.7198443 ,  0.7966285 , -1.0409538 ,
        0.30354515, -0.14173596, -0.9428875 , -0.1314913 , -0.55

In [28]:
sixthsense_vector

array([ 0.02205221,  0.01413373,  0.03755835,  0.00875664,  0.00642459,
       -0.00540978, -0.03847275,  0.00370744, -0.00694549,  0.01522527,
       -0.03075906, -0.03108677, -0.00821988,  0.0324733 ,  0.0204129 ,
        0.00558516, -0.00917993,  0.00787677,  0.02140114,  0.00052991,
        0.0412479 , -0.01407405, -0.00759039,  0.00225028, -0.02144875,
        0.0141123 , -0.01555049,  0.02243233,  0.00759279, -0.0096095 ,
        0.00693261, -0.01141403,  0.01048296, -0.01865646,  0.03223016,
       -0.01887689,  0.05462177, -0.02014632,  0.02627778, -0.0291778 ,
        0.02648038, -0.00745983, -0.00491434, -0.01749343,  0.02851213,
        0.01194941,  0.02030446, -0.01282433, -0.01244222, -0.00456017,
        0.01120098, -0.01696716, -0.00028941,  0.01306434,  0.02208516,
        0.03925216, -0.02183815,  0.01768911, -0.00085477, -0.01096876,
        0.02643718,  0.00775836, -0.02956561,  0.01335054,  0.00258068,
        0.01818206,  0.01405894,  0.00266796, -0.01991807,  0.00

In [29]:
# 나의 선호도 영화와 식스센스, 내적 
np.dot(preference_vector, sixthsense_vector)

0.586615

In [34]:
# Bug's Life에 대한 선호도 예측
bugs_life = movie_to_idx["Bug's Life, A (1998)"]
bugs_life_vector = als_model.item_factors[bugs_life]
np.dot(preference_vector, bugs_life_vector)

0.2178039

In [36]:
# Jurassic Park에 대한 선호도 예측
jurassic_park = movie_to_idx["Jurassic Park (1993)"]
jurassic_park_vector = als_model.item_factors[jurassic_park]
np.dot(preference_vector, jurassic_park_vector)

0.44822252

### 7. 내가 좋아하는 영화와 비슷한 영화를 추천받아보기

In [37]:
def get_similar_artist(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [38]:
get_similar_artist('Sixth Sense, The (1999)')

['Sixth Sense, The (1999)',
 'Silence of the Lambs, The (1991)',
 'batman (1989)',
 'Usual Suspects, The (1995)',
 'Bone Collector, The (1999)',
 'Fugitive, The (1993)',
 'Ghostbusters (1984)',
 'Seven (Se7en) (1995)',
 'Sleepy Hollow (1999)',
 'Fight Club (1999)']

In [39]:
get_similar_artist('batman (1989)')

['batman (1989)',
 'Number Seventeen (1932)',
 'Daens (1992)',
 'War at Home, The (1996)',
 'Century (1993)',
 "Brother's Kiss, A (1997)",
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)',
 "Another Man's Poison (1952)",
 'Male and Female (1919)',
 'Neon Bible, The (1995)']

In [40]:
get_similar_artist('Matrix, The (1999)')

['Matrix, The (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Fugitive, The (1993)',
 'Terminator, The (1984)',
 'batman (1989)',
 'Fifth Element, The (1997)',
 'Jurassic Park (1993)',
 'Men in Black (1997)',
 'Face/Off (1997)']

In [41]:
get_similar_artist('Men in Black (1997)')

['Men in Black (1997)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Independence Day (ID4) (1996)',
 'Matrix, The (1999)',
 'Fifth Element, The (1997)',
 'Lost World: Jurassic Park, The (1997)',
 'batman (1989)',
 'Face/Off (1997)']

In [42]:
get_similar_artist('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Babe (1995)',
 'Aladdin (1992)',
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Pleasantville (1998)',
 'Beauty and the Beast (1991)',
 'batman (1989)']

### 8. 내가 가장 좋아할 만한 영화들을 추천받기

In [43]:
user = user_to_idx['6041']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)

# 영화 제목으로 인덱스 변환
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
recommend_movies = [idx_to_movie[i[0]] for i in movie_recommended]
recommend_movies

['Terminator 2: Judgment Day (1991)',
 'Jurassic Park (1993)',
 'Toy Story 2 (1999)',
 'Fugitive, The (1993)',
 'Total Recall (1990)',
 'Galaxy Quest (1999)',
 'Silence of the Lambs, The (1991)',
 'Babe (1995)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Groundhog Day (1993)',
 'Forrest Gump (1994)',
 'Hunt for Red October, The (1990)',
 "Bug's Life, A (1998)",
 'Fargo (1996)',
 'Lion King, The (1994)',
 'Saving Private Ryan (1998)',
 'American Beauty (1999)',
 'E.T. the Extra-Terrestrial (1982)',
 'Aladdin (1992)',
 'Fifth Element, The (1997)']